<a href="https://colab.research.google.com/github/BonTapHoa/paic-tho-san-hang-e/blob/main/test-15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline: IELTS Writing Task 1 Scoring

Chào mừng quý thí sinh đến với PTNK AI Challenge do The Noders Community tổ chức.

Trong notebook này, chúng ta sẽ xây dựng một mô hình baseline đơn giản để chấm điểm bài thi IELTS Writing Task 1. Mục tiêu là cung cấp một bộ mã nguồn cơ bản để thí sinh nộp kết quả và làm nền tảng cho các cải tiến tiếp theo. Nếu thí sinh là người mới hoàn toàn, vui lòng tham khảo https://docs.google.com/document/d/1545wNRM0TiUealIxDRkfDh8kfnjFATgWdsRPRZTRC0s/edit?usp=sharing để tìm hiểu các khái niệm liên quan.

### Mục tiêu
*   **Input**: Bài văn (Essay) của thí sinh.
*   **Output**: Điểm số (Band Score) từ 1.0 - 9.0.
*   **Đánh giá**: MAE (Mean Absolute Error). Giá trị nhỏ hơn biểu thị hiệu năng tốt hơn.

---
### Hướng dẫn dành cho người mới
Notebook này cung cấp một baseline; thí sinh được khuyến nghị phát triển và tối ưu hoá thêm. Ở mỗi phần có các gợi ý kỹ thuật. Vui lòng đọc kỹ các hướng dẫn này vì chúng giúp thí sinh cải thiện kết quả.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
PATH = '/content/drive/MyDrive/PTNK/PAIC/ielts_7'

In [3]:
# 1. Cài đặt & Import thư viện
# Chúng ta sẽ dùng thư viện Transformers của HuggingFace - tiêu chuẩn vàng trong NLP hiện nay.

import os
import sys
import warnings
import logging

# --- CẤU HÌNH ẨN CÁC WARNING KHÔNG CẦN THIẾT ---
# Tắt log của TensorFlow (nếu có) để giảm thông tin không cần thiết cho người mới
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'
logging.getLogger("transformers").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cài đặt các thư viện cần thiết nếu chưa có
try:
    import transformers
    import datasets
    import gdown
except ImportError:
    print("Đang cài đặt các thư viện: transformers, datasets, accelerate, gdown, scikit-learn...")
    # Thêm -q để cài đặt im lặng hơn
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "transformers", "datasets", "accelerate", "gdown", "scikit-learn"])

import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, TrainingArguments, Trainer
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import EarlyStoppingCallback
from transformers import get_cosine_schedule_with_warmup
from transformers import DataCollatorWithPadding
from transformers import AutoModel, PreTrainedModel, AutoConfig
from transformers.modeling_outputs import SequenceClassifierOutput
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Kiểm tra GPU (Nếu có GPU thì train sẽ nhanh hơn rất nhiều)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Thiết bị sử dụng: {device.upper()}")

Thiết bị sử dụng: CUDA


In [4]:
# 2. Tải dữ liệu từ Google Drive
# Link dữ liệu đã được cung cấp sẵn
train_file_id = '1VLdW_0J7OZdBOPlD-R__JUUCYfGLYpz9'
test_file_id = '1oZioLvy29aLzCZSYALnlYVLSitiQI5Tb'

def download_from_drive(file_id, output_name):
    url = f'https://drive.google.com/uc?id={file_id}'
    if not os.path.exists(output_name):
        print(f"Đang tải {output_name}...")
        gdown.download(url, output_name, quiet=False)
    else:
        print(f"{output_name} đã tồn tại.")

download_from_drive(train_file_id, 'train.csv')
download_from_drive(test_file_id, 'test.csv')

# Đọc dữ liệu
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

print(f"Kích thước tập Train: {df_train.shape}")
print(f"Kích thước tập Test: {df_test.shape}")
display(df_train.head(3))

Đang tải train.csv...


Downloading...
From: https://drive.google.com/uc?id=1VLdW_0J7OZdBOPlD-R__JUUCYfGLYpz9
To: /content/train.csv
100%|██████████| 8.32M/8.32M [00:00<00:00, 64.4MB/s]


Đang tải test.csv...


Downloading...
From: https://drive.google.com/uc?id=1oZioLvy29aLzCZSYALnlYVLSitiQI5Tb
To: /content/test.csv
100%|██████████| 1.64M/1.64M [00:00<00:00, 229MB/s]

Kích thước tập Train: (4517, 9)
Kích thước tập Test: (897, 4)


,ID,prompt,image_description,essay,Overall,TR,CC,LR,GRA
0,0,The charts below show the proportions of Briti...,The image shows two pie charts comparing the p...,The presented pie charts display information a...,6.5,7.5,6.0,6.0,6.0
1,1,The plans below show a public park when it fir...,The image displays two maps of Grange Park sho...,The plans illustrate how Grange Park appeared ...,9.0,9.0,9.0,9.0,9.0
2,2,The graph shows the proportion of four differe...,"The image shows a line graph titled ""Recycling...",The line graph illustrates the percentages of ...,8.0,7.0,8.0,7.0,9.0


### Gợi ý 1: Dữ liệu đầu vào (Data Input)
Trong baseline này, chúng ta **chỉ sử dụng cột `Essay`** để huấn luyện mô hình.
Tuy nhiên, điểm số IELTS Writing Task 1 phụ thuộc nhiều vào việc mô tả đúng biểu đồ.
Gợi ý: Kết hợp cột `Prompt` (đề bài) và `Image Description` (mô tả ảnh) cùng `Essay` để mô hình nắm rõ ngữ cảnh.
Ví dụ: `Input = Prompt + " [SEP] " + Essay`

Chú thích: `SEP` là token phân tách giúp mô hình phân biệt các phần văn bản trong cùng một input.


In [5]:
# 3. Xử lý dữ liệu (Preprocessing) đơn giản
model_checkpoint = "microsoft/deberta-v3-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
sep = tokenizer.sep_token
df_train['text'] = df_train['prompt'] + sep + df_train['image_description'] + sep + df_train['essay']
df_test['text'] = df_test['prompt'] + sep + df_test['image_description'] + sep + df_test['essay']

# Chỉ lấy cột Essay và Overall Score
# Chuẩn hóa tên cột cho dễ làm việc
df_train = df_train.rename(columns={'TR': 'label1', 'CC': 'label2', 'LR': 'label3', 'GRA': 'label4'})
cols_to_combine = ['label1', 'label2', 'label3', 'label4']
df_train['labels'] = df_train[cols_to_combine].values.tolist()

# Chia tập Train thành Train/Validation (80/20) để đánh giá model
train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=42)

print(f"Train size: {len(train_df)}")
print(f"Val size: {len(val_df)}")

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

Train size: 3613
Val size: 904


### Gợi ý 2: Tokenization & Model
Baseline sử dụng **BERT Small** (`prajjwal1/bert-small`). Đây là mô hình kích thước nhỏ (~29 triệu tham số); thời gian chạy nhanh nhưng khả năng biểu diễn ngữ nghĩa hạn chế.
Gợi ý: Hãy thử đổi sang các mô hình mạnh hơn như:
*   `distilbert-base-uncased` (66M params)
*   `microsoft/deberta-v3-base` (86M params - SOTA hiện nay)
*   `roberta-base` (125M params)
Lưu ý: Các mô hình lớn hơn sẽ huấn luyện lâu hơn nhưng thường cho kết quả tốt hơn.


In [6]:
# 4. Tokenization & Dataset
# Hàm tokenize
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Chuyển Pandas DataFrame thành HuggingFace Dataset
train_dataset = datasets.Dataset.from_pandas(train_df[['text', 'labels']])
val_dataset = datasets.Dataset.from_pandas(val_df[['text', 'labels']])
test_dataset = datasets.Dataset.from_pandas(df_test[['text']])

# Áp dụng tokenize
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Xóa các cột text gốc để giải phóng bộ nhớ
tokenized_train = tokenized_train.remove_columns(["text", "__index_level_0__"])
tokenized_val = tokenized_val.remove_columns(["text", "__index_level_0__"])
tokenized_test = tokenized_test.remove_columns(["text"])

# Định dạng cho PyTorch
tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "token_type_ids", "labels"])
tokenized_val.set_format("torch", columns=["input_ids", "attention_mask", "token_type_ids", "labels"])
tokenized_test.set_format("torch", columns=["input_ids", "attention_mask", "token_type_ids"])

print("Đã chuẩn bị xong dataset.")

Map:   0%|          | 0/3613 [00:00<?, ? examples/s]

Map:   0%|          | 0/904 [00:00<?, ? examples/s]

Map:   0%|          | 0/897 [00:00<?, ? examples/s]

Đã chuẩn bị xong dataset.


### 💡 Hint 3: Training Strategy
Trong ví dụ này chỉ huấn luyện **3 epochs** với các tham số mặc định.
👉 **Gợi ý:**
*   Tăng số epoch lên (5-10) và dùng `EarlyStopping` để tránh Overfitting.
*   Tinh chỉnh `learning_rate` (thường trong khoảng 2e-5 đến 5e-5).
*   Áp dụng **K-Fold Cross Validation** (chia dữ liệu thành 5 phần và huấn luyện 5 mô hình) để có kết quả ổn định hơn so với huấn luyện một lần.

(Phần này thuộc mức nâng cao; ban tổ chức chưa chuẩn bị tài liệu chi tiết về nội dung này. Thí sinh có thể tham khảo tài liệu tham khảo bên ngoài hoặc đặt câu hỏi trên kênh trao đổi của cuộc thi.)

Giải thích thêm cho những bạn chưa quen: sau khi chạy cell huấn luyện phía dưới, kết quả đánh giá sẽ hiển thị cột `MAE`. Với mô hình minh họa do ban tổ chức sử dụng, MAE tham khảo thường nằm trong khoảng 0.8–1.0. Giá trị này cho biết sai số trung bình của dự đoán (khoảng 0.8–1.0 band). Kết quả có thể được cải thiện nếu áp dụng các gợi ý ở trên hoặc thử nghiệm thêm các phương pháp nâng cao.

In [ ]:
# @title
# 5. Khởi tạo Model & Trainer
def get_optimizer_grouped_parameters(model, learning_rate, weight_decay, layer_decay=0.9):
    """
    Phân nhóm tham số để áp dụng LLRD.
    Layer trên cùng (Classifier) nhận learning_rate gốc.
    Các layer thấp hơn nhận: lr * (layer_decay ^ depth)
    """

    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = []

    # --- 1. NHÓM CLASSIFIER & POOLER (HEAD) ---
    # Nhóm này quan trọng nhất để fix Underfitting -> LR cao nhất
    head_params = []
    for name, param in model.named_parameters():
        if "classifier" in name or "pooler" in name:
            head_params.append((name, param))

    optimizer_grouped_parameters.extend([
        {
            "params": [p for n, p in head_params if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
            "lr": learning_rate,
        },
        {
            "params": [p for n, p in head_params if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
            "lr": learning_rate,
        },
    ])

    # --- 2. NHÓM BACKBONE (DeBERTa Layers) ---
    # DeBERTa V3 base có 12 lớp
    if hasattr(model, "bert"):
        backbone = model.bert
    else:
        # Fallback nếu bạn đặt tên biến khác trong class IELTSModel
        backbone = model.deberta

    num_layers = backbone.config.num_hidden_layers

    for layer_i in range(num_layers - 1, -1, -1):
        layer_name = f"encoder.layer.{layer_i}."

        # Công thức decay: LR lớp i = LR gốc * (decay ^ (khoảng cách tới đỉnh))
        current_lr = learning_rate * (layer_decay ** (num_layers - layer_i))

        layer_params = []
        for name, param in model.named_parameters():
            if layer_name in name and ("bert" in name or "deberta" in name):
                layer_params.append((name, param))

        optimizer_grouped_parameters.extend([
            {
                "params": [p for n, p in layer_params if not any(nd in n for nd in no_decay)],
                "weight_decay": weight_decay,
                "lr": current_lr,
            },
            {
                "params": [p for n, p in layer_params if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
                "lr": current_lr,
            },
        ])

    # --- 3. NHÓM EMBEDDINGS (Đáy cùng) ---
    embedding_lr = learning_rate * (layer_decay ** (num_layers + 1))
    embedding_params = []
    for name, param in model.named_parameters():
        if "embeddings" in name and ("bert" in name or "deberta" in name):
            embedding_params.append((name, param))

    optimizer_grouped_parameters.extend([
        {
            "params": [p for n, p in embedding_params if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
            "lr": embedding_lr,
        },
        {
            "params": [p for n, p in embedding_params if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
            "lr": embedding_lr,
        },
    ])

    return optimizer_grouped_parameters

class AttentionPooling(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(in_dim, in_dim),
            nn.LayerNorm(in_dim),
            nn.GELU(),
            nn.Linear(in_dim, 1)
        )

    def forward(self, last_hidden_state, attention_mask):
        w = self.attention(last_hidden_state)
        if attention_mask is not None:
            padding_mask = attention_mask.unsqueeze(-1)
            w = w.masked_fill(padding_mask == 0, float('-inf'))
        w = torch.softmax(w, dim=1)
        context_vector = torch.sum(w * last_hidden_state, dim=1)
        return context_vector

class IELTSModel(nn.Module):
    def __init__(self, checkpoint, num_labels=1):
        super(IELTSModel, self).__init__()
        self.bert = AutoModel.from_pretrained(checkpoint)
        hidden_size = self.bert.config.hidden_size
        self.pooler = AttentionPooling(hidden_size)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 512),
            nn.LayerNorm(512),
            nn.Dropout(0.5),
            nn.Linear(512, num_labels)
        )

        self.loss_fct = nn.L1Loss()

    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        feature_vector = self.pooler(last_hidden_state, attention_mask)
        logits = self.classifier(feature_vector)
        loss = None
        if labels is not None:
            loss = self.loss_fct(logits, labels.float())
        return SequenceClassifierOutput(loss=loss, logits=logits)

model = IELTSModel(model_checkpoint, num_labels=4)
model.to(device)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Định nghĩa hàm tính metric MAE
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    mae = mean_absolute_error(labels, predictions)
    return {"mae": mae}

# --- CẤU HÌNH LLRD ---
learning_rate = 5e-5
weight_decay = 0.1
layer_decay = 0.8     # Mỗi lớp giảm 10% learning rate so với lớp trên nó

optimizer_grouped_parameters = get_optimizer_grouped_parameters(
    model,
    learning_rate,
    weight_decay,
    layer_decay
)

optimizer = optim.AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=1e-8)
num_epochs = 30
batch_size = 16
gradient_accumulation_steps = 64 // batch_size

# Tổng số bước = (Số mẫu train / batch_size) * số epochs
num_train_steps = (len(tokenized_train) // batch_size) * num_epochs

lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=int(0.1 * num_train_steps), # 10% warmup
    num_training_steps=num_train_steps
)

# 5. Cấu hình Arguments
args = TrainingArguments(
    output_dir=PATH,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=learning_rate, # Tham số này chỉ để log, thực tế dùng optimizer truyền vào
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=num_epochs,
    weight_decay=weight_decay,
    load_best_model_at_end=True,
    metric_for_best_model="mae",
    greater_is_better=False,
    save_total_limit=2,
    report_to='none',
    fp16=True,
)

# 6. Khởi tạo Trainer với Optimizer & Scheduler tự tạo
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    optimizers=(optimizer, lr_scheduler),
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

print("Bắt đầu huấn luyện...")
trainer.train()
print("Huấn luyện hoàn tất.")

Bắt đầu huấn luyện...


Epoch,Training Loss,Validation Loss,Mae
1,5.757400,4.642624,4.642624
2,2.847300,1.384467,1.384467
3,1.287000,1.071518,1.071517
4,1.134000,0.948285,0.948285
5,1.053900,0.871772,0.871772
6,0.956900,0.809466,0.809466
7,0.869800,0.709685,0.709685
8,0.822800,0.671784,0.671784
9,0.815200,0.723971,0.723971
10,0.755200,0.624408,0.624408


### Gợi ý 4: Post-processing & Ensemble
Kết quả dự đoán của model là số thực (ví dụ: 6.342).
Tuy nhiên, điểm IELTS thường được làm tròn (ví dụ: 6.0, 6.5, 7.0).
Gợi ý:
*   Viết hàm làm tròn điểm về thang 0.5 gần nhất.
*   **Ensemble:** Train nhiều model khác nhau (DistilBERT, DeBERTa, RoBERTa) rồi lấy trung bình cộng kết quả dự đoán. Đây là cách hiệu quả nhất để giảm MAE!


In [9]:
# @title
import numpy as np

def pre_processing(arr):
    """
    Hàm xử lý hậu kỳ cho mô hình 4 labels (TR, CC, LR, GRA).
    Input: Mảng numpy kích thước (N, 4) hoặc (N*4,)
    Output: Mảng numpy kích thước (N,) chứa điểm Overall đã làm tròn.
    """
    # 1. Kiểm tra kích thước dữ liệu
    if arr.ndim == 1:
        arr = arr.reshape(-1, 4)
    overall_raw = np.mean(arr, axis=1)
    y_pred = np.round(overall_raw * 2) / 2
    y_pred = np.clip(y_pred, 1.0, 9.0)
    return y_pred

# 6. Dự đoán trên tập Test & Tạo Submission
predictions = trainer.predict(tokenized_test)
y_pred = pre_processing(predictions.predictions)
if 'id' in df_test.columns:
    ids = df_test['id']
else:
    print("Không tìm thấy cột 'id' trong file test. Sử dụng index làm id...")
    ids = df_test.index

submission = pd.DataFrame({
    'id': ids,
    'Overall': y_pred
})

# Hiển thị vài mẫu để kiểm tra
print(submission.head())

# Lưu file csv
submission.to_csv(PATH + '/submission.csv', index=False)
print("Đã lưu file submission.csv. Vui lòng nộp file này để kiểm tra kết quả.")

Không tìm thấy cột 'id' trong file test. Sử dụng index làm id...
   id  Overall
0   0      4.5
1   1      8.5
2   2      7.0
3   3      7.5
4   4      6.5
Đã lưu file submission.csv. Vui lòng nộp file này để kiểm tra kết quả.


# LOAD MODEL

In [ ]:
# @title
import torch
import torch.nn as nn
import os
from transformers import AutoModel, AutoTokenizer
from transformers.modeling_outputs import SequenceClassifierOutput
try:
    from safetensors.torch import load_file
except ImportError:
    print("Đang cài đặt thư viện safetensors...")
    os.system("pip install safetensors")
    from safetensors.torch import load_file

# --- PHẦN 1: ĐỊNH NGHĨA MODEL (Giữ nguyên) ---
class AttentionPooling(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(in_dim, in_dim),
            nn.LayerNorm(in_dim),
            nn.GELU(),
            nn.Linear(in_dim, 1)
        )

    def forward(self, last_hidden_state, attention_mask):
        w = self.attention(last_hidden_state)
        if attention_mask is not None:
            padding_mask = attention_mask.unsqueeze(-1)
            w = w.masked_fill(padding_mask == 0, float('-inf'))
        w = torch.softmax(w, dim=1)
        context_vector = torch.sum(w * last_hidden_state, dim=1)
        return context_vector

class IELTSModel(nn.Module):
    def __init__(self, checkpoint, num_labels=1):
        super(IELTSModel, self).__init__()
        self.bert = AutoModel.from_pretrained(checkpoint)
        hidden_size = self.bert.config.hidden_size
        self.pooler = AttentionPooling(hidden_size)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 512),
            nn.LayerNorm(512),
            nn.Dropout(0.1),
            nn.Linear(512, num_labels)
        )
        self.loss_fct = nn.L1Loss()

    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        feature_vector = self.pooler(last_hidden_state, attention_mask)
        logits = self.classifier(feature_vector)
        loss = None
        if labels is not None:
            loss = self.loss_fct(logits, labels.float())
        return SequenceClassifierOutput(loss=loss, logits=logits)

# --- PHẦN 2: KHỞI TẠO VÀ LOAD WEIGHTS ---

# Định nghĩa đường dẫn thư mục gốc chứa checkpoint
CHECKPOINT_DIR = "/content/drive/MyDrive/PTNK/PAIC/ielts_5/checkpoint-6780"

# 1. Khởi tạo kiến trúc
print("Đang khởi tạo kiến trúc model...")
model_checkpoint = "microsoft/deberta-v3-base"
model = IELTSModel(model_checkpoint, num_labels=4)

# 2. Load trọng số (Cập nhật logic nhận diện file)
weight_path = os.path.join(CHECKPOINT_DIR, "pytorch_model.bin")
use_safetensors = False

if not os.path.exists(weight_path):
    weight_path = os.path.join(CHECKPOINT_DIR, "model.safetensors")
    if os.path.exists(weight_path):
        use_safetensors = True

print(f"Đang load weights từ: {weight_path}")

try:
    if use_safetensors:
        # Dùng thư viện safetensors để load
        state_dict = load_file(weight_path, device="cpu")
    else:
        # Dùng torch để load file bin truyền thống
        state_dict = torch.load(weight_path, map_location=torch.device('cpu'), weights_only=False)

    model.load_state_dict(state_dict)
    print("✅ Đã load model thành công!")
except Exception as e:
    print(f"❌ Lỗi khi load weights: {e}")
    # Gợi ý debug nếu vẫn lỗi
    if not os.path.exists(weight_path):
        print(f"⚠️ Kiểm tra lại đường dẫn, file không tồn tại: {weight_path}")

# 3. Chuyển sang chế độ dự đoán
model.eval()

# --- PHẦN 3: LOAD TOKENIZER ---
print(f"Đang load tokenizer từ thư mục: {CHECKPOINT_DIR}")
try:
    tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT_DIR)
    print("✅ Đã load tokenizer thành công!")
except Exception as e:
    print(f"❌ Lỗi tokenizer: {e}")

Đang khởi tạo kiến trúc model...
Đang load weights từ: /content/drive/MyDrive/PTNK/PAIC/ielts_5/checkpoint-6780/model.safetensors
❌ Lỗi khi load weights: could not determine the shape of object type 'torch.storage.UntypedStorage'
Đang load tokenizer từ thư mục: /content/drive/MyDrive/PTNK/PAIC/ielts_5/checkpoint-6780
✅ Đã load tokenizer thành công!


In [ ]:
# @title
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

# 1. Hàm xử lý hậu kỳ (Giữ nguyên của bạn)
def pre_processing(arr):
    """
    Hàm xử lý hậu kỳ cho mô hình 4 labels (TR, CC, LR, GRA).
    Input: Mảng numpy kích thước (N, 4) hoặc (N*4,)
    Output: Mảng numpy kích thước (N,) chứa điểm Overall đã làm tròn.
    """
    if arr.ndim == 1:
        arr = arr.reshape(-1, 4)
    overall_raw = np.mean(arr, axis=1)
    y_pred = np.round(overall_raw * 2) / 2
    y_pred = np.clip(y_pred, 1.0, 9.0)
    return y_pred

# 2. Hàm dự đoán thủ công (Thay thế cho trainer.predict)
def predict_manual(model, tokenized_dataset, batch_size=16):
    # Tạo DataLoader để chia nhỏ dữ liệu thành các batch
    dataloader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=False)

    # Xác định thiết bị (GPU hoặc CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval() # Chuyển model sang chế độ đánh giá

    all_logits = []

    print("Đang chạy dự đoán...")
    with torch.no_grad(): # Tắt tính toán gradient để tiết kiệm bộ nhớ
        for batch in tqdm(dataloader):
            # Chuyển dữ liệu sang thiết bị (GPU/CPU)
            inputs = {k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'attention_mask', 'token_type_ids']}

            # Forward pass
            outputs = model(**inputs)
            logits = outputs.logits

            # Chuyển kết quả về CPU và lưu lại
            all_logits.append(logits.cpu().numpy())

    # Nối tất cả các batch lại thành một mảng lớn
    return np.concatenate(all_logits, axis=0)

# --- THỰC THI ---

# Kiểm tra xem biến tokenized_test và df_test có tồn tại không
# (Bạn phải chạy các cell xử lý dữ liệu phía trên Notebook trước cell này)
try:
    # 6. Dự đoán trên tập Test
    raw_predictions = predict_manual(model, tokenized_test)

    # Xử lý hậu kỳ
    y_pred = pre_processing(raw_predictions)

    # Tạo DataFrame Submission
    if 'id' in df_test.columns:
        ids = df_test['id']
    else:
        print("Không tìm thấy cột 'id' trong file test. Sử dụng index làm id...")
        ids = df_test.index

    submission = pd.DataFrame({
        'id': ids,
        'Overall': y_pred
    })

    # Hiển thị và lưu file
    print(submission.head())
    submission.to_csv('/content/drive/MyDrive/PTNK/PAIC/ielts_5/checkpoint-6780/submission.csv', index=False)
    print("\n✅ Đã lưu file submission.csv thành công!")

except NameError as e:
    print(f"❌ Lỗi thiếu biến dữ liệu: {e}")
    print("👉 Bạn cần chạy lại các cell 'Load dữ liệu' và 'Tokenization' (Cell 2, 3, 4 trong notebook gốc) để tạo lại biến 'df_test' và 'tokenized_test'.")

Đang chạy dự đoán...


  0%|          | 0/57 [00:00<?, ?it/s]

Không tìm thấy cột 'id' trong file test. Sử dụng index làm id...
   id  Overall
0   0      4.5
1   1      9.0
2   2      6.5
3   3      6.5
4   4      6.5

✅ Đã lưu file submission.csv thành công!


# LOAD MODEL TRAIN TIẾP

In [8]:
import torch
import torch.optim as optim
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import get_cosine_schedule_with_warmup, EarlyStoppingCallback
import torch
import torch.nn as nn
import os
from transformers import AutoModel, AutoTokenizer
from transformers.modeling_outputs import SequenceClassifierOutput
try:
    from safetensors.torch import load_file
except ImportError:
    print("Đang cài đặt thư viện safetensors...")
    os.system("pip install safetensors")
    from safetensors.torch import load_file

# --- 1. CẤU HÌNH LẠI CÁC THAM SỐ (Phải giống hệt lúc train cũ) ---
# Đường dẫn đến checkpoint bạn đang có trên Drive
CHECKPOINT_DIR = "/content/drive/MyDrive/PTNK/PAIC/ielts_7/checkpoint-1254"

# Các tham số Hyperparameters (Lấy từ notebook gốc)
learning_rate = 5e-5
weight_decay = 0.1
layer_decay = 0.8
batch_size = 16
num_epochs = 40
gradient_accumulation_steps = 64 // batch_size

# --- 2. TẠO LẠI OPTIMIZER & SCHEDULER (Vì notebook dùng LLRD custom) ---
# Hàm phân nhóm tham số (LLRD) - Copy y chang từ notebook gốc
def get_optimizer_grouped_parameters(model, learning_rate, weight_decay, layer_decay=0.9):
    """
    Phân nhóm tham số để áp dụng LLRD.
    Layer trên cùng (Classifier) nhận learning_rate gốc.
    Các layer thấp hơn nhận: lr * (layer_decay ^ depth)
    """

    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = []

    # --- 1. NHÓM CLASSIFIER & POOLER (HEAD) ---
    # Nhóm này quan trọng nhất để fix Underfitting -> LR cao nhất
    head_params = []
    for name, param in model.named_parameters():
        if "classifier" in name or "pooler" in name:
            head_params.append((name, param))

    optimizer_grouped_parameters.extend([
        {
            "params": [p for n, p in head_params if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
            "lr": learning_rate,
        },
        {
            "params": [p for n, p in head_params if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
            "lr": learning_rate,
        },
    ])

    # --- 2. NHÓM BACKBONE (DeBERTa Layers) ---
    # DeBERTa V3 base có 12 lớp
    if hasattr(model, "bert"):
        backbone = model.bert
    else:
        # Fallback nếu bạn đặt tên biến khác trong class IELTSModel
        backbone = model.deberta

    num_layers = backbone.config.num_hidden_layers

    for layer_i in range(num_layers - 1, -1, -1):
        layer_name = f"encoder.layer.{layer_i}."

        # Công thức decay: LR lớp i = LR gốc * (decay ^ (khoảng cách tới đỉnh))
        current_lr = learning_rate * (layer_decay ** (num_layers - layer_i))

        layer_params = []
        for name, param in model.named_parameters():
            if layer_name in name and ("bert" in name or "deberta" in name):
                layer_params.append((name, param))

        optimizer_grouped_parameters.extend([
            {
                "params": [p for n, p in layer_params if not any(nd in n for nd in no_decay)],
                "weight_decay": weight_decay,
                "lr": current_lr,
            },
            {
                "params": [p for n, p in layer_params if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
                "lr": current_lr,
            },
        ])

    # --- 3. NHÓM EMBEDDINGS (Đáy cùng) ---
    embedding_lr = learning_rate * (layer_decay ** (num_layers + 1))
    embedding_params = []
    for name, param in model.named_parameters():
        if "embeddings" in name and ("bert" in name or "deberta" in name):
            embedding_params.append((name, param))

    optimizer_grouped_parameters.extend([
        {
            "params": [p for n, p in embedding_params if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
            "lr": embedding_lr,
        },
        {
            "params": [p for n, p in embedding_params if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
            "lr": embedding_lr,
        },
    ])

    return optimizer_grouped_parameters

class AttentionPooling(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(in_dim, in_dim),
            nn.LayerNorm(in_dim),
            nn.GELU(),
            nn.Linear(in_dim, 1)
        )

    def forward(self, last_hidden_state, attention_mask):
        w = self.attention(last_hidden_state)
        if attention_mask is not None:
            padding_mask = attention_mask.unsqueeze(-1)
            w = w.masked_fill(padding_mask == 0, float('-inf'))
        w = torch.softmax(w, dim=1)
        context_vector = torch.sum(w * last_hidden_state, dim=1)
        return context_vector

class IELTSModel(nn.Module):
    def __init__(self, checkpoint, num_labels=1):
        super(IELTSModel, self).__init__()
        self.bert = AutoModel.from_pretrained(checkpoint)
        hidden_size = self.bert.config.hidden_size
        self.pooler = AttentionPooling(hidden_size)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 512),
            nn.LayerNorm(512),
            nn.Dropout(0.5),
            nn.Linear(512, num_labels)
        )

        self.loss_fct = nn.L1Loss()

    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        feature_vector = self.pooler(last_hidden_state, attention_mask)
        logits = self.classifier(feature_vector)
        loss = None
        if labels is not None:
            loss = self.loss_fct(logits, labels.float())
        return SequenceClassifierOutput(loss=loss, logits=logits)

model_checkpoint = "microsoft/deberta-v3-large"
model = IELTSModel(model_checkpoint, num_labels=4)
model.to(device)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 1. Khởi tạo kiến trúc
print("Đang khởi tạo kiến trúc model...")

# 2. Load trọng số (Cập nhật logic nhận diện file)
weight_path = os.path.join(CHECKPOINT_DIR, "pytorch_model.bin")
use_safetensors = False

if not os.path.exists(weight_path):
    weight_path = os.path.join(CHECKPOINT_DIR, "model.safetensors")
    if os.path.exists(weight_path):
        use_safetensors = True

print(f"Đang load weights từ: {weight_path}")

try:
    if use_safetensors:
        # Dùng thư viện safetensors để load
        state_dict = load_file(weight_path, device="cpu")
    else:
        # Dùng torch để load file bin truyền thống
        state_dict = torch.load(weight_path, map_location=torch.device('cpu'), weights_only=False)

    model.load_state_dict(state_dict)
    print("✅ Đã load model thành công!")
except Exception as e:
    print(f"❌ Lỗi khi load weights: {e}")
    # Gợi ý debug nếu vẫn lỗi
    if not os.path.exists(weight_path):
        print(f"⚠️ Kiểm tra lại đường dẫn, file không tồn tại: {weight_path}")

# --- PHẦN 3: LOAD TOKENIZER ---
print(f"Đang load tokenizer từ thư mục: {CHECKPOINT_DIR}")
try:
    tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT_DIR)
    print("✅ Đã load tokenizer thành công!")
except Exception as e:
    print(f"❌ Lỗi tokenizer: {e}")

# Định nghĩa hàm tính metric MAE
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    mae = mean_absolute_error(labels, predictions)
    return {"mae": mae}

optimizer_grouped_parameters = get_optimizer_grouped_parameters(
    model,
    learning_rate,
    weight_decay,
    layer_decay
)

optimizer = optim.AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=1e-8)
# Tổng số bước = (Số mẫu train / batch_size) * số epochs
num_train_steps = (len(tokenized_train) // batch_size) * num_epochs

lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=int(0.1 * num_train_steps), # 10% warmup
    num_training_steps=num_train_steps
)

# 5. Cấu hình Arguments
args = TrainingArguments(
    output_dir=PATH,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=learning_rate, # Tham số này chỉ để log, thực tế dùng optimizer truyền vào
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=num_epochs,
    weight_decay=weight_decay,
    load_best_model_at_end=True,
    metric_for_best_model="mae",
    greater_is_better=False,
    save_total_limit=2,
    report_to='none',
    fp16=True,
)

# 6. Khởi tạo Trainer với Optimizer & Scheduler tự tạo
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    optimizers=(optimizer, lr_scheduler),
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

# --- 4. BẮT ĐẦU RESUME ---
print(f"Đang tiếp tục train từ checkpoint: {CHECKPOINT_DIR}")
# resume_from_checkpoint=True sẽ tìm checkpoint mới nhất trong output_dir
# Hoặc truyền đường dẫn cụ thể để chắc chắn
try:
    trainer.train(resume_from_checkpoint=CHECKPOINT_DIR)
    print("Huấn luyện tiếp hoàn tất!")
except ValueError as e:
    print(f"Lỗi resume: {e}")
    print("Gợi ý: Kiểm tra xem trong folder checkpoint có file 'optimizer.pt' và 'scheduler.pt' chưa?")
    print("Nếu không có, hãy set `resume_from_checkpoint=False` để train lại từ đầu với weights hiện tại (nhưng optimizer sẽ bị reset).")

Đang khởi tạo kiến trúc model...
Đang load weights từ: /content/drive/MyDrive/PTNK/PAIC/ielts_7/checkpoint-1254/model.safetensors
✅ Đã load model thành công!
Đang load tokenizer từ thư mục: /content/drive/MyDrive/PTNK/PAIC/ielts_7/checkpoint-1254
✅ Đã load tokenizer thành công!
Đang tiếp tục train từ checkpoint: /content/drive/MyDrive/PTNK/PAIC/ielts_7/checkpoint-1254


Epoch,Training Loss,Validation Loss,Mae
23,0.560500,0.610300,0.610300
24,0.566800,0.570463,0.570463
25,0.539400,0.604476,0.604476
26,0.536200,0.614390,0.614390
27,0.531600,0.606364,0.606364
28,0.526700,0.584755,0.584755
29,0.506500,0.586603,0.586604
30,0.501200,0.589592,0.589592
31,0.504600,0.605227,0.605227
32,0.495400,0.604544,0.604544


Huấn luyện tiếp hoàn tất!
